In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from abc import abstractmethod, ABC
from keras.optimizers import Adam
from tensorflow.keras import layers
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
class MnistClassifierInterface(ABC):
  @abstractmethod
  def train(self, X, y):
    pass

  @abstractmethod
  def predict(self, X):
    pass

In [ ]:
class RFClassifier(MnistClassifierInterface):
  def __init__(self, n_estimators=100, criterion="gini", max_depth=None):
    self.model = RandomForestClassifier(n_estimators=n_estimators,
                                        criterion=criterion,
                                        max_depth=max_depth)

  def train(self, X, y):
    self.model.fit(X, y)
    y_pred = self.model.predict(X)
    train_acc = accuracy_score(y, y_pred)
    print("Train accuracy: ", train_acc)

  def predict(self, X):
    return self.model.predict(X)

In [ ]:
class FeedForwardNNClassifier(MnistClassifierInterface):
  def __init__(self, X_size=None, y_size=None, epochs=10,
               batch_size=32, lr=0.0001):
    self.lr = lr
    self.epochs = epochs
    self.batch_size = batch_size

    self.model = keras.Sequential([
        layers.Input(shape=X_size),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(y_size, activation="softmax"),
    ])
    self.model.compile(optimizer=Adam(learning_rate=self.lr),
                       loss="sparse_categorical_crossentropy",
                       metrics=["accuracy"])

  def train(self, X, y):
    self.model.fit(X, y, epochs=self.epochs,
                         batch_size=self.batch_size)

  def predict(self, X):
    y_pred = self.model.predict(X)
    return np.argmax(y_pred, axis=1)

In [ ]:
class CNNClassifier(MnistClassifierInterface):
  def __init__(self, X_size=None, y_size=None, epochs=10,
               batch_size=32, lr=0.0001):
    self.lr = lr
    self.epochs = epochs
    self.batch_size = batch_size

    self.model = keras.Sequential([
        layers.Lambda(lambda x: tf.expand_dims(x, -1), input_shape=X_size),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(y_size, activation="softmax"),
    ])
    self.model.compile(optimizer=Adam(learning_rate=self.lr),
                       loss="sparse_categorical_crossentropy",
                       metrics=["accuracy"])

  def train(self, X, y):
    self.model.fit(X, y, epochs=self.epochs,
                         batch_size=self.batch_size)

  def predict(self, X):
    y_pred = self.model.predict(X)
    return np.argmax(y_pred, axis=1)

In [ ]:
class MnistClassifier:
  def __init__(self, alg_type, **kwargs):
    if alg_type == "cnn":
      self.model = CNNClassifier(**kwargs)
    elif alg_type == "rf":
      self.model = RFClassifier(**kwargs)
    elif alg_type == "nn":
      self.model = FeedForwardNNClassifier(**kwargs)
    else:
      raise ValueError("There is no such algorithm. Please select one of the presented algorithms: cnn, rf, nn")

  def train(self, X, y):
    self.model.train(X, y)

  def predict(self, X):
    return self.model.predict(X)

In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

X_train: (60000, 28, 28), X_test: (10000, 28, 28)
y_train: (60000,), y_test: (10000,)


In [ ]:
# Data normalization
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.

# Flatten images for RF and FeedForward NN model
X_train_flatten = X_train.reshape((len(X_train), np.prod(X_train.shape[1:])))
X_test_flatten = X_test.reshape((len(X_test), np.prod(X_test.shape[1:])))

In [ ]:
# RandomForest model test
model = MnistClassifier(alg_type="rf")
model.train(X_train_flatten, y_train)

y_test_pred = model.predict(X_test_flatten)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test accuracy: ", test_acc)

Train accuracy:  1.0
Test accuracy:  0.9688


In [ ]:
# FeedForward NN model test
model = MnistClassifier(alg_type="nn", X_size=(X_train_flatten.shape[1],), y_size=10)
model.train(X_train_flatten, y_train)

y_test_pred = model.predict(X_test_flatten)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test accuracy: ", test_acc)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7176 - loss: 1.0459
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9260 - loss: 0.2630
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9443 - loss: 0.2005
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9533 - loss: 0.1634
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9608 - loss: 0.1385
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9652 - loss: 0.1224
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9696 - loss: 0.1081
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9734 - loss: 0.0967
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.9752 - loss: 0.0884
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9778 - loss: 0.0797
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Test accuracy:  0.9705


In [ ]:
# CNN model test
model = MnistClassifier(alg_type="cnn", X_size=(28, 28), y_size=10)
model.train(X_train, y_train)

y_test_pred = model.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test accuracy: ", test_acc)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 54s 28ms/step - accuracy: 0.6177 - loss: 1.2369
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - accuracy: 0.9224 - loss: 0.2564
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 54s 29ms/step - accuracy: 0.9471 - loss: 0.1714
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 52s 28ms/step - accuracy: 0.9585 - loss: 0.1384
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - accuracy: 0.9613 - loss: 0.1226
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 28ms/step - accuracy: 0.9695 - loss: 0.1031
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 28ms/step - accuracy: 0.9706 - loss: 0.0967
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 51s 27ms/step - accuracy: 0.9747 - loss: 0.0838
Epoch 9/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 27ms/step - accuracy: 0.9753 - loss: 0.0804
Epoch 10/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 82s 27ms/step - accuracy: 0.9766 - loss: 0.0749
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Test accuracy:  0.9859
